In [1]:
pwd

'C:\\Users\\SERGHI\\Documents\\GitHub\\Diplomado_PUCP\\Lecture_9\\Assignment_7'

In [2]:
import pandas as pd

health_estb = pd.read_excel(r'../../_data/cajamarca_data_helth_estb.xlsx')

districts = pd.read_excel(r'../../_data/peru_districts_centroids.xlsx')

cajamarca_districts = districts[districts['NOMBDEP'] == 'CAJAMARCA']

In [3]:
import pandas as pd
from tqdm import tqdm
import numpy as np

# Crear un nuevo DataFrame para almacenar los resultados
result_df = cajamarca_districts.copy()

# Crear nuevas columnas en 'result_df' para almacenar el establecimiento de salud más cercano
result_df['closest_health_estb'] = np.nan
result_df['closest_health_estb_lat'] = np.nan
result_df['closest_health_estb_lon'] = np.nan

# Iterar sobre cada fila en 'result_df'
for index, district_row in tqdm(result_df.iterrows(), total=len(result_df), desc="Finding closest health establishment"):
    district_coords = (district_row['Centroid_Latitude'], district_row['Centroid_Longitude'])
    
    # Calcular la distancia euclidiana para cada establecimiento de salud y encontrar el más cercano
    min_distance = float('inf')
    closest_health_estb = None
    closest_health_estb_lat = None
    closest_health_estb_lon = None
    
    for _, health_row in health_estb.iterrows():
        health_coords = (health_row['latitud'], health_row['longitud'])
        distance = np.linalg.norm(np.array(district_coords) - np.array(health_coords))
        
        if distance < min_distance:
            min_distance = distance
            closest_health_estb = health_row['nombre']
            closest_health_estb_lat = health_row['latitud']
            closest_health_estb_lon = health_row['longitud']
    
    # Almacenar el establecimiento de salud más cercano en las nuevas columnas
    result_df.at[index, 'closest_health_estb'] = closest_health_estb
    result_df.at[index, 'closest_health_estb_lat'] = closest_health_estb_lat
    result_df.at[index, 'closest_health_estb_lon'] = closest_health_estb_lon

# Ahora 'result_df' contiene los resultados sin afectar 'cajamarca_districts'


Finding closest health establishment: 100%|███████████████████████████████████████████████████████| 127/127 [00:32<00:00,  3.95it/s]
